In [1]:
import fiftyone as fo
import os
import numpy as np
from utils.utils import create_yolov4_dataset
from utils.utils import create_yolov5_dataset


In [2]:
# The directory containing the source images
data_path = "/Users/madhuhegde/work/berkeley/W210/leopard.coco/images/train2022/"

# The path to the COCO labels JSON file
labels_path = "/Users/madhuhegde/work/berkeley/W210/leopard.coco/annotations/instances_train2022.json"

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path
)

 100% |███████████████| 6795/6795 [11.1s elapsed, 0s remaining, 672.4 samples/s]      


In [ ]:

def create_yolov5_dataset(dataset, train_test_split = 0.8, output_folder='./'):
    anno = [0, 0, 0, 0, 0]
    train_files = []
    test_files = []
       
    np.random.seed(42)
    
    train_image_path =  output_folder+'/train'                   
    isExist = os.path.exists(train_image_path)                  
                    
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(train_image_path+'/images') 
        os.makedirs(train_image_path+'/labels')  
                        
    test_image_path =  output_folder+'/test'                       
    isExist = os.path.exists(test_image_path)

    if not isExist:
        os.makedirs(test_image_path+'/images') 
        os.makedirs(test_image_path+'/labels')                    

    for sample in dataset:  
        anno[1:] = sample['ground_truth']['detections'][0]['bounding_box']
        height = sample['metadata']['height']
        width = sample['metadata']['width']
        
        
        if anno[3] >=1.0:
            anno[3] = 1-1.0/width
            
        if anno[4] >=1.0:
            anno[4] = 1-1.0/height    

        anno[1] = anno[1]+0.5*anno[3]
        anno[2] = anno[2]+0.5*anno[4]       
     
        old_image_file = sample['filepath']
       
        if(np.random.random_sample() > train_test_split):        
            new_image_file = train_image_path + '/images/'+ old_image_file.split("/")[-1]
            new_anno_file = train_image_path + '/labels/'+ old_image_file.split("/")[-1]
                
        else:
            new_image_file =  test_image_path + '/images/'+ old_image_file.split("/")[-1]     
            new_anno_file = test_image_path + '/labels/'+ old_image_file.split("/")[-1]
                        
        new_anno_file = new_anno_file.replace("jpg", "txt")              
           
        copy_str = 'cp ' + old_image_file + ' ' + new_image_file
        os.system(copy_str)
        
        with open(new_anno_file, 'w') as fp:
            for item in anno:
            # write each item on a new line
                fp.write("%s " % item)
            fp.write("\n")  
    
   
    

In [3]:
#create_yolov5_dataset(dataset, output_folder="/home/user1/work/darknet/yolov4/darknet")
create_yolov5_dataset(dataset, output_folder="/Users/madhuhegde/work/berkeley/W210/Animal_Identification/datasets/leopard")

In [ ]:
data_path = "/content/darknet/leopard.coco/images/train2022"
# The path to the COCO labels JSON file
labels_path = "/content/darknet/leopard.coco/annotations/instances_train2022.json"
# Import the COCO dataset
dataset1 = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path,
) 

In [ ]:

for sample in dataset:
  path = sample.filepath[:-3] + 'txt'
  with open(path, 'w') as f:
    anno = sample['ground_truth']['detections'][0]['bounding_box']
    height = sample['metadata']['height']
    width = sample['metadata']['width']
    
    if anno[2] >=1.0:
        anno[2] = 1-1.0/width
        
    if anno[3] >=1.0:
        anno[3] = 1-1.0/height
    anno[0] = anno[0] + 0.5*anno[2]
    anno[1] = anno[1] + 0.5*anno[3]
    bb = str(anno[0]) + ' ' + str(anno[1]) + ' ' + str(anno[2]) + ' ' + str(anno[3])
    f.write('0 ' + bb)
    
    
# Import the YOLO dataset
dataset2 = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv4Dataset,
    data_path=data_path,
)

In [ ]:
session = fo.launch_app(dataset1)

In [ ]:
session2 = fo.launch_app(dataset2)